<a href="https://colab.research.google.com/github/harshitadd/CrossSiloFLDemo/blob/master/PreVID_PySyft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install syft --quiet 

In [ ]:
##Importing Mini Batch Data 
import os 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pydicom --quiet
import pydicom 
import cv2
import matplotlib.pyplot as plt 

     |████████████████████████████████| 35.5MB 89kB/s 


In [ ]:
dcm_path=os.listdir('/content/drive/My Drive/Fed_Covid/minibatch/')
dcm_data={}

alpha = 1.5 # Contrast control (1.0-3.0)
beta = 0 # Brightness control (0-100)

ctr=0
for file in dcm_path:
  name = '/content/drive/My Drive/Fed_Covid/minibatch/' + file
  temp = pydicom.dcmread(name)
  image = temp.pixel_array
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  adjusted = cv2.resize(image,(64,64))
  dcm_data[file]=adjusted
  ctr+=1

In [ ]:
import csv 
labels=[]
with open('/content/drive/My Drive/Fed_Covid/stage_2_train_labels.csv','r') as file:
  reader = csv.reader(file)
  for row in reader:
    labels.append(row)

In [ ]:
from sklearn.preprocessing import MinMaxScaler 
import numpy as np
scaler = MinMaxScaler()
pid=[]
dicom=[]
label=[]
cid = 0
for PID in labels:
  for key in dcm_data:
    if(key[:-4]==PID[0]):
      l=[]
      for val in dcm_data[key]:
        l.append(scaler.fit_transform(val))
      l = np.reshape(l,(3,64,64))
      dicom.append(l)
      label.append(int(PID[5]))
      

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import syft as sy
from torch.utils.data import TensorDataset, DataLoader
import time
import copy
import numpy as np
from torch.utils.data import Dataset
#from syft.frameworks.torch.federated import utils
from syft.frameworks.torch.fl import utils
from syft.workers.websocket_client import WebsocketClientWorker

In [ ]:
class Arguments():
    def __init__(self):
        self.batch_size = 4
        self.test_batch_size = 4
        self.epochs = 5
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 10
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(dicom,label,test_size=0.3)

In [ ]:
import pandas as pd 
train_df = pd.DataFrame()
train_df['features']=x_train
train_df['labels']=y_train

In [ ]:
train_df

,features,labels
0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",0
1,"[[[0.011299435028248588, 0.011299435028248588,...",0
2,"[[[0.014285714285714287, 0.014285714285714287,...",0
3,"[[[0.027586206896551724, 0.027586206896551724,...",0
4,"[[[0.19069767441860463, 0.19069767441860463, 0...",0
...,...,...
295,"[[[0.23204419889502761, 0.23204419889502761, 0...",0
296,"[[[0.026595744680851064, 0.026595744680851064,...",0
297,"[[[0.13846153846153847, 0.13846153846153847, 0...",0
298,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",0


In [ ]:
test_df = pd.DataFrame()
test_df['features']=x_test
test_df['labels']=y_test

In [ ]:
# train_data, train_labels = np.array(train_df.features, dtype=np.float16),np.array(train_df.labels,dtype=np.float16)  

In [ ]:
# class data_maker(Dataset):
#   def __init__(self, features, labels,transform = None):
#         self.data = features 
#         self.targets = labels 
#         self.to_torchtensor()
        
#   def __len__(self):
#       return len(self.data)

#   def to_torchtensor(self):
#     self.data=torch.from_numpy(self.data)
#     self.labels=torch.from_numpy(self.targets)
 
  
#   def __getitem__(self, idx):
#         sample=np.array(self.data[idx])
#         target=np.array(self.targets[idx])
                
#         return sample,target

In [ ]:
class data_maker(Dataset):
  def __init__(self, data):
        self.data = data
        
  def __len__(self):
      return len(self.data)
  
  def __getitem__(self, index):
      target = np.array(self.data['labels'][index])
      data_val = np.array(self.data['features'][index])

      return data_val,target
      
        

In [ ]:
train_data = data_maker(train_df)
test_data = data_maker(test_df)

In [ ]:
train_data

In [ ]:
federated_data = data_maker(train_df).federate((hospital,clinic))

In [ ]:
federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
                         federated_data,batch_size=args.batch_size)

In [ ]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        model.send(data.location) # <-- NEW: send the model to the right location
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get() # <-- NEW: get the model back
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_loader) * args.batch_size, #batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [ ]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [ ]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment

for epoch in range(1, args.epochs + 1):
    train(args, model, device, federated_train_loader, optimizer, epoch)
    test(args, model, device, test_loader)



RuntimeError: ignored

In [ ]:
def get_train_data(batch_size):
      train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=False)
      return train_loader

In [ ]:
train_loader= get_train_data(batch_size=4)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=4)

In [ ]:
n_batches = len(train_loader)
for i, data in enumerate(train_loader, 0):
       inputs, labels = data
       print(np.shape(inputs))
       print(np.shape(labels))
       break   
for i, data in enumerate(test_loader, 0):
       inputs, labels = data
       print(np.shape(inputs))
       print(np.shape(labels))
       break   

torch.Size([4, 3, 64, 64])
torch.Size([4])
torch.Size([4, 3, 64, 64])
torch.Size([4])


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        # Input - (32,1024,1024)
  
        self.conv1 = nn.Conv2d(3,32, kernel_size=8, stride = 2)  #I/P - 1*32
        self.pool1 = nn.MaxPool2d(kernel_size=4,stride=2,padding=0)  
        self.conv2 = nn.Conv2d(32, 64, 8)
        self.pool2 = nn.MaxPool2d(4,2,padding=0)
        self.fc1 = nn.Linear(256, 64*64)
        self.fc2 = nn.Linear(64*64, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        
        x = F.relu(self.conv1(x))
        # Input of 1*1024*1024 going to 32*1024*1024  - 1*64*64 going to 32*64*64
        x = self.pool1(x)
        # Input of 32*1024*1024 going to 32*256*256 - 32*64*64 going to 32*16*16
        x = F.relu(self.conv2(x))
        # Input of 32*256*256 to 64*256*256 - 32*16*16 to 64*16*16
        x = self.pool2(x)
        # Input of 64*256*256 to 64*64*64 - 64*16*16 going to 64*4*4

        x = x.view(-1,64*4)
     
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return F.log_softmax(x,dim=1)

In [ ]:
def createLossAndOptimizer(net, learning_rate=0.001):
    
    #Loss function
    loss = torch.nn.CrossEntropyLoss()
    
    #Optimizer
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    
    return(loss, optimizer)

https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html - Test Loader

PYTORCH IMPLEMENTATION


In [ ]:
import syft as sy 
hook = sy.TorchHook(torch)

In [ ]:
hospital = sy.VirtualWorker(hook, id="hospital")  
clinic = sy.VirtualWorker(hook, id="clinic")  
# me = hook.local_worker
# me.is_client_worker = False


In [ ]:
model= Net().double()

In [ ]:
class Arguments():
    def __init__(self):
        self.batch_size = 4
        self.test_batch_size = 10
        self.epochs = 5
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 1
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}


In [ ]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader): # <-- now it is a distributed dataset
        model.send(data.location) # <-- NEW: send the model to the right location
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.mse_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get() # <-- NEW: get the model back
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_loader) * args.batch_size, #batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


In [ ]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.mse_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment

for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

AttributeError: ignored

In [ ]:
# me.add_workers([hospital, clinic])
# clinic.add_workers([me, hospital])
# hospital.add_workers([me, clinic])

In [ ]:
compute_nodes = [hospital, clinic]

In [ ]:
remote_dataset = (list(), list())
from torch.autograd import Variable

In [ ]:
train_distributed_dataset = []
for batch_idx, (data,target) in enumerate(train_loader):
    data = Variable(data)
    target = Variable(target.double())
    data = data.send(compute_nodes[batch_idx % len(compute_nodes)])
    target = target.send(compute_nodes[batch_idx % len(compute_nodes)])
    remote_dataset[batch_idx % len(compute_nodes)].append((data, target))
    # data.send(compute_nodes[batch_idx % len(compute_nodes)])
    # target.send(compute_nodes[batch_idx % len(compute_nodes)])
    # train_distributed_dataset.append((data, target))


In [ ]:
model = Net().double()

In [ ]:
model

Net(
  (conv1): Conv2d(3, 32, kernel_size=(8, 8), stride=(2, 2))
  (pool1): MaxPool2d(kernel_size=4, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(8, 8), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=4, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=256, out_features=4096, bias=True)
  (fc2): Linear(in_features=4096, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=8, bias=True)
)

In [ ]:
def train(epoch):

    model.train()
    for batch_idx, (data,target) in enumerate(train_distributed_dataset):           
        
        worker = data.location
        print(worker)
        model.send(worker)

        optimizer.zero_grad()
        # update the model
        pred = model(data)
        loss = F.mse_loss(pred, target.double())
        loss.backward()
        model.get()
        optimizer.step()

        loss.get()
        print('Train Epoch:' + str(epoch) + str(loss.data[0]))
    


In [ ]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    for data, target in test_loader:

        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.mse_loss(output, target.double(), size_average=False).data[0] # sum up batch loss
        _, predicted = torch.max(output.data, 1)
        #pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}\n'.format(test_loss))

In [ ]:
for epoch in range(5):
    train(epoch)

In [ ]:
hospital_model = Net().double()
clinic_model = Net().double()

hospital_optimizer = optim.SGD(hospital_model.parameters(), lr=0.001)
clinic_optimizer = optim.SGD(clinic_model.parameters(), lr=0.001)

models = [hospital_model, clinic_model]
optimizers = [hospital_optimizer, clinic_optimizer]

model = Net().double()
model

Net(
  (conv1): Conv2d(3, 32, kernel_size=(8, 8), stride=(2, 2))
  (pool1): MaxPool2d(kernel_size=4, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(8, 8), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=4, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=256, out_features=4096, bias=True)
  (fc2): Linear(in_features=4096, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=1, bias=True)
)

In [ ]:
def update(data, target, model, optimizer):
    print('reached')
    model.send(data.location)
    optimizer.zero_grad()
    prediction = model(data)
    print(np.shape(prediction))
    print(np.shape(target))
    loss = F.mse_loss(prediction.view(4), target.double())
    loss.backward()
    optimizer.step()
    return model


In [ ]:
def train():
    for data_index in range(len(remote_dataset[0])-1):
        for remote_index in range(len(compute_nodes)):
            data, target = remote_dataset[remote_index][data_index]
            # print(np.shape(data)) - 4*3*64*64
            # print(np.shape(target)) - 4 
            models[remote_index] = update(data, target.double(), models[remote_index], optimizers[remote_index])
        for model in models:
            model.get()
        return utils.federated_avg({
            "hospital": models[0],
            "clinic": models[1]
        })


In [ ]:
def test(federated_model):
    federated_model.eval()
    test_loss = 0
    for data, target in test_loader:
        output = federated_model(data)
        print('test')
        print(np.shape(output))
        print(np.shape(target))
        test_loss += F.mse_loss(output, target.double()).data
        #test_loss += F.mse_loss(output, target.double(), reduction='sum').item()
        _, predicted = torch.max(output.data, 1)
        #pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        
        #predection = output.data.max(1, keepdim=True)[1]
        
    test_loss /= len(test_loader.dataset)
    print('Test set: Average loss: {:.4f}'.format(test_loss))


In [ ]:
for epoch in range(5):
    start_time = time.time()
    print(f"Epoch Number {epoch + 1}")
    federated_model = train()
    model = federated_model
    test(federated_model)
    total_time = time.time() - start_time
    print('Communication time over the network', round(total_time, 2), 's\n')

Epoch Number 1
reached
torch.Size([4, 1])
torch.Size([4])
reached
torch.Size([4, 1])
torch.Size([4])
test
torch.Size([4, 1])
torch.Size([4])
test
torch.Size([4, 1])
torch.Size([4])
test
torch.Size([4, 1])
torch.Size([4])
test
torch.Size([4, 1])
torch.Size([4])
test
torch.Size([4, 1])
torch.Size([4])
test
torch.Size([4, 1])
torch.Size([4])
test
torch.Size([4, 1])
torch.Size([4])
test
torch.Size([4, 1])
torch.Size([4])
test
torch.Size([4, 1])
torch.Size([4])
test
torch.Size([4, 1])
torch.Size([4])
test
torch.Size([4, 1])
torch.Size([4])
test
torch.Size([4, 1])
torch.Size([4])
test
torch.Size([4, 1])
torch.Size([4])
test
torch.Size([4, 1])
torch.Size([4])
test
torch.Size([4, 1])
torch.Size([4])
test
torch.Size([4, 1])
torch.Size([4])


/usr/local/lib/python3.6/dist-packages/syft/frameworks/torch/tensors/interpreters/native.py:401: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  response = command_method(*args_, **kwargs_)


test
torch.Size([4, 1])
torch.Size([4])
test
torch.Size([4, 1])
torch.Size([4])
test
torch.Size([4, 1])
torch.Size([4])
test
torch.Size([4, 1])
torch.Size([4])
test
torch.Size([4, 1])
torch.Size([4])
test
torch.Size([4, 1])
torch.Size([4])
test
torch.Size([4, 1])
torch.Size([4])
test
torch.Size([4, 1])
torch.Size([4])
test
torch.Size([4, 1])
torch.Size([4])
test
torch.Size([4, 1])
torch.Size([4])
test
torch.Size([4, 1])
torch.Size([4])
test
torch.Size([4, 1])
torch.Size([4])
test
torch.Size([4, 1])
torch.Size([4])
test
torch.Size([4, 1])
torch.Size([4])
test
torch.Size([4, 1])
torch.Size([4])
test
torch.Size([4, 1])
torch.Size([4])
test
torch.Size([1, 1])
torch.Size([1])
Test set: Average loss: 0.0484
Communication time over the network 1.09 s

Epoch Number 2


/usr/local/lib/python3.6/dist-packages/syft/frameworks/torch/tensors/interpreters/native.py:401: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  response = command_method(*args_, **kwargs_)


ObjectNotFoundError: ignored